In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import joblib

# ==========================================
# PART 1: GENERATE SESSION DATA
# ==========================================
print("1. Generating Session Data for LSTM...")

# Map specific actions to numbers (Neural Networks need numbers)
# This represents the "vocabulary" of our app
ACTIONS = {
    'LOGIN': 1,
    'VIEW_BALANCE': 2,
    'VIEW_TRANSACTIONS': 3,
    'TRANSFER_SMALL': 4,   # < $500 (Normal behavior)
    'TRANSFER_LARGE': 5,   # > $10,000 (Risky)
    'CHANGE_PASSWORD': 6,
    'ADD_RECIPIENT': 7,
    'LOGOUT': 8
}

# --- A. Generate NORMAL Sessions ---
# Pattern: Login -> Check stuff -> Maybe small transfer -> Logout
# We generate 10,000 normal sessions to simulate real traffic
normal_sessions = []
for _ in range(10000):
    # Random session length between 3 and 8 actions
    seq_len = np.random.randint(3, 8)
    session = [ACTIONS['LOGIN']]
    
    for _ in range(seq_len):
        # Normal users mostly view balance/transactions (80% chance)
        action_code = np.random.choice(
            [ACTIONS['VIEW_BALANCE'], ACTIONS['VIEW_TRANSACTIONS'], ACTIONS['TRANSFER_SMALL'], ACTIONS['LOGOUT']], 
            p=[0.4, 0.4, 0.1, 0.1]
        )
        session.append(action_code)
        if action_code == ACTIONS['LOGOUT']:
            break
            
    normal_sessions.append(session)

# --- B. Generate ATTACK Sessions (Account Takeover) ---
# Pattern: Login -> Change Pass/Add Recipient -> Big Transfer -> Logout
# These sequences are distinctively "aggressive"
attack_sessions = []

# Attack Pattern 1: The "Quick Drain"
# Login -> Add Recipient -> Transfer ALL Money
for _ in range(500):
    session = [ACTIONS['LOGIN'], ACTIONS['ADD_RECIPIENT'], ACTIONS['TRANSFER_LARGE'], ACTIONS['LOGOUT']]
    attack_sessions.append(session)

# Attack Pattern 2: The "Password Reset"
# Login -> Change Password -> Transfer
for _ in range(500):
    session = [ACTIONS['LOGIN'], ACTIONS['CHANGE_PASSWORD'], ACTIONS['TRANSFER_LARGE'], ACTIONS['LOGOUT']]
    attack_sessions.append(session)

print(f"   Normal Sessions Generated: {len(normal_sessions)}")
print(f"   Attack Sessions Generated: {len(attack_sessions)}")

# ==========================================
# PART 2: PREPROCESSING
# ==========================================
print("\n2. Preprocessing sequences...")

# LSTMs require all sequences to be the same length. 
# We "pad" short sessions with 0s to make them all length 10.
MAX_SEQ_LENGTH = 10 
X_normal = pad_sequences(normal_sessions, maxlen=MAX_SEQ_LENGTH, padding='post')
X_attack = pad_sequences(attack_sessions, maxlen=MAX_SEQ_LENGTH, padding='post')

# Create Labels (0 = Normal, 1 = Attack)
# We use Supervised Learning here because we know exactly what a "bad sequence" looks like
y_normal = np.zeros(len(X_normal))
y_attack = np.ones(len(X_attack))

# Combine and Split
X = np.concatenate([X_normal, X_attack])
y = np.concatenate([y_normal, y_attack])

# Split 80% for Training, 20% for Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# PART 3: BUILD THE LSTM MODEL
# ==========================================
print("\n3. Building LSTM Architecture...")
# As per PDF: "Analyzes the order and timing of events"

model = Sequential()

# 1. Embedding Layer
# Turns integers (1, 2, 3...) into dense vectors.
# input_dim = 9 (8 actions + 0 padding)
# output_dim = 32 (Each action is represented by 32 numbers)
model.add(Embedding(input_dim=9, output_dim=32, input_length=MAX_SEQ_LENGTH))

# 2. LSTM Layer
# 64 units of memory to remember previous actions in the sequence
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2)) # Prevents overfitting (memorizing specific examples)

# 3. Output Layer
# Sigmoid gives a probability between 0 and 1
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ==========================================
# PART 4: TRAIN
# ==========================================
print("\n4. Training Model...")
# We train for 15 epochs. 
# Validation data checks if the model actually "Learned" or just "Memorized".
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

# Save Model
model.save("model_lstm.h5")
print("   LSTM Model Saved as 'model_lstm.h5'")

# ==========================================
# PART 5: TEST REAL-TIME PREDICTION
# ==========================================
print("\n5. Testing Live Sequences...")

def predict_session_risk(action_sequence):
    # 1. Convert text actions to numbers
    seq_nums = [ACTIONS.get(a, 0) for a in action_sequence]
    # 2. Pad sequence to length 10
    padded_seq = pad_sequences([seq_nums], maxlen=MAX_SEQ_LENGTH, padding='post')
    # 3. Predict Probability
    risk_score = model.predict(padded_seq, verbose=0)[0][0]
    return risk_score

# Test Case 1: Normal User checking bank
seq1 = ['LOGIN', 'VIEW_BALANCE', 'VIEW_TRANSACTIONS', 'LOGOUT']
risk1 = predict_session_risk(seq1)
print(f"Sequence: {seq1}")
print(f"Risk Score: {risk1:.4f} [{'🔴 BLOCKED' if risk1 > 0.8 else '🟢 ALLOWED'}]")

# Test Case 2: Hacker trying to drain account
seq2 = ['LOGIN', 'CHANGE_PASSWORD', 'TRANSFER_LARGE']
risk2 = predict_session_risk(seq2)
print(f"\nSequence: {seq2}")
print(f"Risk Score: {risk2:.4f} [{'🔴 BLOCKED' if risk2 > 0.8 else '🟢 ALLOWED'}]")

# Test Case 3: Hacker trying to add recipient first
seq3 = ['LOGIN', 'ADD_RECIPIENT', 'TRANSFER_LARGE']
risk3 = predict_session_risk(seq3)
print(f"\nSequence: {seq3}")
print(f"Risk Score: {risk3:.4f} [{'🔴 BLOCKED' if risk3 > 0.8 else '🟢 ALLOWED'}]")

1. Generating Session Data for LSTM...
   Normal Sessions Generated: 10000
   Attack Sessions Generated: 1000

2. Preprocessing sequences...

3. Building LSTM Architecture...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 32)            288       
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 25,185
Trainable params: 25,185
Non-trainable params: 0
_________________________________________________________________

4. Train